# 7.4 word2vec、fastTextを用いた日本語単語のベクトル表現の実装

- 本ファイルでは、日本語の単語をword2vecもしくはfastTextを使用してベクトル化する手法を解説します。

※　本章のファイルはすべてUbuntuでの動作を前提としています。Windowsなど文字コードが違う環境での動作にはご注意下さい。

# 7.4 学習目標

1.	学習済みの日本語word2vecモデルで単語をベクトル表現に変換する実装ができるようになる
2.	学習済みの日本語fastText モデルで単語をベクトル表現に変換する実装ができるようになる


# 事前準備
書籍の指示に従い、本章で使用するデータを用意します

pip install gensim


# 1. 文書を読み込んで、分かち書き、データセット作成まで（8.2と同じです）

前処理と分かち書きをし、最後にデータセットを作成する部分を実装します


In [1]:
# 単語分割にはMecabを使用
import MeCab

m_t = MeCab.Tagger('-Ochasen -d /usr/lib/mecab/dic/mecab-ipadic-neologd')

def tokenizer_mecab(text):
    text = m_t.parse(text)  # これでスペースで単語が区切られる
    ret = text.strip().split()  # スペース部分で区切ったリストに変換
    return ret



# 前処理として正規化をする関数を定義
import re

def preprocessing_text(text):
    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)

    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 ０-９]', '0', text)  # 数字

    return text


# 前処理とJanomeの単語分割を合わせた関数を定義する


def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)  # 前処理の正規化
    ret = tokenizer_mecab(text)  # Mecabの単語分割

    return ret


In [2]:
import torchtext

# tsvやcsvデータを読み込んだときに、読み込んだ内容に対して行う処理を定義します
# 文章とラベルの両方に用意します

max_length = 25
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                            use_vocab=True, lower=True, include_lengths=True, batch_first=True, fix_length=max_length)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)


# フォルダ「data」から各tsvファイルを読み込みます
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='text_train.tsv',
    validation='text_val.tsv', test='text_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# 2. 単語のベクトル化

## 2.1 word2vec

単語をベクトル表現に変換します。

TorchTextには日本語の学習済みデータがないわけではないですが、精度が微妙なので

東北大学 乾・岡崎研究室で公開されているWord2Vecの学習済みのベクトルを使用します。



In [ ]:
# 以下から、日本語のfasttextの学習済みベクトルをダウンロードします

# 東北大学 乾・岡崎研究室：日本語 Wikipedia エンティティベクトル

# http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/
# http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2

In [ ]:
# そのままではtorchtextで読み込めないので、gensimライブラリを使用して、
# Word2Vecのformatで保存し直します

# 事前インストール
# pip install gensim

from gensim.models import KeyedVectors


# 一度gensimライブラリで読み込んで、word2vecのformatで保存する
model = KeyedVectors.load_word2vec_format(
    './data/entity_vector/entity_vector.model.bin', binary=True)

# 保存（時間がかかります、10分弱）
model.wv.save_word2vec_format('./data/japanese_word2vec_vectors.vec')


In [13]:
# torchtextで単語ベクトルとして読み込みます
from torchtext.vocab import Vectors

japanese_word2vec_vectors = Vectors(
    name='./data/japanese_word2vec_vectors.vec')

# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", japanese_word2vec_vectors.dim)
print("単語数：", len(japanese_word2vec_vectors.itos))


1単語を表現する次元数： 200
単語数： 1015474


In [14]:
# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=japanese_word2vec_vectors, min_freq=1)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 52個の単語が200次元のベクトルで表現されている
TEXT.vocab.vectors


torch.Size([52, 200])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 2.6023, -2.6357, -2.5822,  ...,  0.6953, -1.4977,  1.4752],
        ...,
        [-1.5885,  0.1614, -0.6029,  ..., -1.7545, -1.2462,  2.3034],
        [-0.3807,  0.4007, -1.6304,  ..., -2.2620, -0.5581,  0.9880],
        [-3.2000,  2.9676, -0.9007,  ...,  1.8192, -0.4641,  0.3200]])

In [15]:
# ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi


defaultdict(<function torchtext.vocab._default_unk_index()>,
            {'<unk>': 0,
             '<pad>': 1,
             'と': 2,
             '。': 3,
             'な': 4,
             'の': 5,
             '文章': 6,
             '、': 7,
             'が': 8,
             'し': 9,
             'を': 10,
             'いる': 11,
             'か': 12,
             'て': 13,
             'ます': 14,
             '分類': 15,
             '本章': 16,
             '評価': 17,
             '0': 18,
             'い': 19,
             'から': 20,
             'する': 21,
             'その': 22,
             'た': 23,
             'で': 24,
             'です': 25,
             'に': 26,
             'に対して': 27,
             'は': 28,
             'まし': 29,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '処理': 36,
             '取り組み': 37,
             '商品': 38,
             '女性': 39,
             '女王': 40,
   

In [16]:
# 姫 - 女性 + 男性 のベクトルがどれと似ているのか確認してみます
import torch.nn.functional as F

# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[42] - \
    TEXT.vocab.vectors[39] + TEXT.vocab.vectors[48]

# コサイン類似度を計算
# dim=0 は0次元目で計算してくださいという指定
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[40], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[46], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[47], dim=0))
print("機械", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[45], dim=0))


女王 tensor(0.3840)
王 tensor(0.3669)
王子 tensor(0.5489)
機械 tensor(-0.0867)


姫 - 女性 + 男性　を計算すると狙った通り、王子がもっとも近い結果になりました

## 2.2 fastText

word2vecより進歩したベクトル化手法であるfastTextによる単語のベクトル表現を使用します。

日本語の学習モデルを以下の記事にて公開してくださっているので、使用させていただきます。



In [ ]:
# Qiita：いますぐ使える単語埋め込みベクトルのリスト
# https://qiita.com/Hironsan/items/8f7d35f0a36e0f99752c

# Download Word Vectors
# https://drive.google.com/open?id=0ByFQ96A4DgSPNFdleG1GaHcxQzA

In [8]:
# torchtextで単語ベクトルとして読み込みます
# word2vecとは異なり、すぐに読み込めます

from torchtext.vocab import Vectors

japanese_fasttext_vectors = Vectors(name='data/vector_neologd/model.vec')

                                    
# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", japanese_fasttext_vectors.dim)
print("単語数：", len(japanese_fasttext_vectors.itos))


100%|█████████▉| 350330/351122 [00:42<00:00, 7758.02it/s]

1単語を表現する次元数： 300
単語数： 351122


100%|█████████▉| 350330/351122 [01:00<00:00, 7758.02it/s]

In [11]:
# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=japanese_fasttext_vectors, min_freq=1)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 52個の単語が300次元のベクトルで表現されている
TEXT.vocab.vectors

# ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi


torch.Size([52, 300])


defaultdict(<function torchtext.vocab._default_unk_index()>,
            {'<unk>': 0,
             '<pad>': 1,
             'と': 2,
             '。': 3,
             'な': 4,
             'の': 5,
             '文章': 6,
             '、': 7,
             'が': 8,
             'し': 9,
             'を': 10,
             'いる': 11,
             'か': 12,
             'て': 13,
             'ます': 14,
             '分類': 15,
             '本章': 16,
             '評価': 17,
             '0': 18,
             'い': 19,
             'から': 20,
             'する': 21,
             'その': 22,
             'た': 23,
             'で': 24,
             'です': 25,
             'に': 26,
             'に対して': 27,
             'は': 28,
             'まし': 29,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '処理': 36,
             '取り組み': 37,
             '商品': 38,
             '女性': 39,
             '女王': 40,
   

In [12]:
# 姫 - 女性 + 男性 のベクトルがどれと似ているのか確認してみます
import torch.nn.functional as F

# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[42] - \
    TEXT.vocab.vectors[39] + TEXT.vocab.vectors[48]

# コサイン類似度を計算
# dim=0 は0次元目で計算してくださいという指定
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[40], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[46], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[47], dim=0))
print("機械", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[45], dim=0))


女王 tensor(0.3650)
王 tensor(0.3461)
王子 tensor(0.5531)
機械 tensor(0.1078)


姫 - 女性 + 男性　を計算すると狙った通り、王子がもっとも近い結果になりました

以上